In [80]:
# 부품 크기만큼 자르기
# 참고 : http://www.gisdeveloper.co.kr/?p=6570
import cv2
import numpy as np

image = cv2.imread("overkill_test5.jpg")
image = cv2.resize(image, dsize=(700,600))
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (3, 3), 0)
edged = cv2.Canny(blurred, 10, 50)

# 모폴로지 구조 커널 생성
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

# 이미지 팽창
dilate = cv2.dilate(edged, kernel, iterations=1)

# 팽창한 이미지에서 컨투어 찾기
contours, _ = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
img = image.copy()

# 면적이 큰 컨투어 찾기
contour = []
for cnt in contours:
    contour.append(cv2.contourArea(cnt))
max_cnt = contours[contour.index(max(contour))]
    
print(contour, contours[contour.index(max(contour))])

# 이미지에 면적이 큰 컨투어 그리기
cv2.drawContours(img, [max_cnt], -1, (0, 255, 0), 2)
print(len(contours), "objects were found in this image.")

# 부품 크기만큼 사각형 그린 후 잘라내기
x, y, w, h = cv2.boundingRect(max_cnt)
img = cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
cropped = img[y:y+h,x+35:x+w-40]
cropped2 = img[y+45:y+h-45,x+85:x+w-90]

# 이미지 띄우기
cv2.imshow("max contours", img)
cv2.imshow("cropped", cropped)
cv2.imshow("cropped2", cropped2)
cv2.imwrite('cropped.jpg',cropped2)
cv2.waitKey(0)
cv2.destroyAllWindows()

[4197.0, 34.5, 725.5, 328934.0, 22.0, 3014.0] [[[626  33]]

 [[625  34]]

 [[617  34]]

 ...

 [[634  34]]

 [[632  34]]

 [[631  33]]]
6 objects were found in this image.


In [83]:
# 노출된 bump 검출 코드
## 템플릿은 잘린 이미지를 기준으로 만들어야함
## 센서면 + bump 부분까지 잘라야함
# 참고
# https://mandloh.tistory.com/97
# https://towardsdatascience.com/object-detection-on-python-using-template-matching-ab4243a0ca62

# 모듈 로딩
import cv2
import numpy as np

# 이미지 불러오기
img = cv2.imread("cropped_over6.jpg")
template_h = cv2.imread("template_h.jpg")   # 가로면 템플릿
template_v = cv2.imread("template_v.jpg")   # 세로면 템플릿

# 그레이스케일화
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
template_h = cv2.cvtColor(template_h, cv2.COLOR_BGR2GRAY)
template_v = cv2.cvtColor(template_v, cv2.COLOR_BGR2GRAY)

# 이미지 스무딩
img_gray = cv2.GaussianBlur(img_gray, (3, 3), 0)

# 템플릿 사이즈를 변수에 저장
w, h = template_h.shape[::-1]
w2, h2 = template_v.shape[::-1]

# 템플릿 매칭
# TM_CCOEFF_NORMED : 최소값이 아닌 최대값을 가져와야하기 때문에 사용함, 즉 하나가 아닌 여러개 찾기 위함
res_h = cv2.matchTemplate(img_gray, template_h, cv2.TM_CCOEFF_NORMED)  # 가로 체크
res_v = cv2.matchTemplate(img_gray, template_v, cv2.TM_CCOEFF_NORMED)  # 세로 체크
threshold = 0.91  # 0 ~ 1의 값, 높을수록 정확한 결과

# 가로, 세로면 각각 불량 탐색
# 가로면에서 불량 탐색
if np.where(res_h >= threshold):
    loc = np.where(res_h >= threshold)  # res_h 중 threshold보다 큰 값 위치 저장
    cnt_h = 0
    for pt in zip(*loc[::-1]):
        # 결과값에 사각형 그리기
        cv2.rectangle(img, pt, (pt[0] + w, pt[1] + h + 10), (0, 0, 255), 1)
        cnt_h += 1

# 세로면에서 불량 탐색
if np.where(res_v >= threshold):
    loc = np.where(res_v >= threshold)  # res_v 중 threshold보다 큰 값 위치 저장
    cnt_v = 0
    for pt in zip(*loc[::-1]):
        # 결과값에 사각형 그리기
        cv2.rectangle(img, pt, (pt[0] + w2 + 5, pt[1] + h2 + 10), (0, 0, 255), 1)
        cnt_v += 1
cnt = cnt_h + cnt_v

# 이미지에 결과 텍스트 삽입
print("총 불량 부위 :", cnt)
if cnt >= 1:
    cv2.putText(img, "NG", (10, 30), cv2.FONT_ITALIC, 1, (0, 0, 255), 2)
else:
    cv2.putText(img, "OK", (10, 30), cv2.FONT_ITALIC, 1, (0, 255, 0), 2)

# 검사 결과 이미지 출력
cv2.imshow("result", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

총 불량 부위 : 0
[[-0.09601106  0.04304044  0.1522583  ...  0.10665302  0.16287427
   0.28557527]
 [ 0.01796035  0.19378549  0.21344449 ...  0.20829616  0.43059102
   0.5507703 ]
 [ 0.31870458  0.36938998  0.08946554 ... -0.00769219  0.20745651
   0.3643542 ]
 ...
 [-0.07004314 -0.01391295 -0.02310114 ... -0.01519306 -0.02304447
  -0.05315024]
 [-0.17236364 -0.11431649 -0.06293354 ... -0.0396448  -0.0324726
  -0.02527991]
 [-0.17269784 -0.12395598 -0.06037259 ... -0.07470733 -0.07041721
  -0.04578997]] [[-0.56019086 -0.5317936  -0.42141977 ... -0.18561308 -0.29911724
  -0.19557045]
 [-0.07692636 -0.22968267 -0.37593043 ... -0.42935446 -0.47349703
  -0.2329701 ]
 [ 0.38286602  0.25481653  0.05408783 ... -0.40114647 -0.3706403
  -0.15639234]
 ...
 [ 0.45479253  0.2558069   0.08363969 ...  0.03523663  0.02617473
  -0.01561342]
 [ 0.3379717   0.25507188  0.18655796 ... -0.08057632 -0.07940426
  -0.08783249]
 [ 0.17619233  0.20506436  0.24207218 ... -0.11410021 -0.11461995
  -0.0936067 ]] (434, 